# Conoco Phillips Dataset Challenge - Team J.J.I.S




The initial portion of our code is import statements and our data cleaning function. We experimented with a variety of ways to process the data, mainly due to the issue of large amounts of 'na's being found within. One of the methods implemented was replacing all entries of 'na' with -1 , we also tried dropping all columns above a certain 'na' threshold. In the end we found that a threshold of 79% resulted in the highest score. This threshold was decided based on sensors 41, 42, and 43 being sequential while also having a similar amount of 'na' percentage, around 80%. 

We attempted a few methods of normalization within the data, such as standard and robust normalization. They were implemented using the standard scikit library functions StandardScaler and RobustScaler. However, we found that normalizing the data resulted in lower accuracy.  

In [6]:
import pandas as pd
import numpy as np
import settings
import os
os.chdir("../Data")
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
# from sklearn import neighbors
from sklearn.metrics import f1_score
import seaborn as sns
import collections


def clean_data(df_clean):
    # Drop columns with greater than 30% of values equal to na
    columns = list(df_clean)
    for col in columns:
        try:
            if df_clean[col].str.count('na').sum() > (60001 * .30):
                df_clean = df_clean.drop(columns=[col])
        except:
            pass
    # Count na's in row and drop rows with greater than 30%
    df_clean['sum_na'] = df_clean.apply(lambda row: sum(row[0:60000]=='na'), axis=1)
    df_done = df_clean[~(df_clean['sum_na'] >= (172 * .3))]
    df_done = df_done.drop(columns=['sum_na'])
    df_done = df_done.replace('na', -1)

    return df_done

We tested machine learning algorithms such as SVM, Random Forest and AdaBoost. For the majority of experiments, random forest gave the best performance regardless of the cleaning or normalization method implemented. For random forest

Another technique we attempted was non-negative matrix factorization. We converted all 'na's to -1, then shifted the entire dataset up by 1, resulting in 'na' being 0. We then converted it to an embedding space using scikit learns MF algorithm. The model was then stored as a pickled object and fed into our random forest algorithm.

In [ ]:
import pickle
from sklearn.decomposition import NMF

if __name__ == "__main__":
    settings.init()

    df_training = pd.read_csv(settings.training_data_cleaned)
    df_drop = df_training.drop(columns=['id','target'])
    df_drop = df_drop + 1

    np_drop = df_drop.to_numpy()
    model = NMF(n_components=2, init='random', random_state=0)
    W = model.fit_transform(np_drop)

    with open('new_matrix.pickle', 'wb') as handle:
        pickle.dump(handle, W)

In [7]:
def gen_output(predictions):
    # columns = ['id', 'target']
    pass


if __name__ == "__main__":
    settings.init()
    print("Executing...")

    df = pd.read_csv(settings.training_data)
    X = df.iloc[:, 2:]  # First two columns are id and target
    Y = np.array(df.iloc[:, 1])

    # Algorithms
    # knn = neighbors.KNeighborsClassifier(n_neighbors=5)
    alg = RandomForestClassifier(n_estimators=100)
    # sv = svm.SVC(kernel='linear')
    # ada = AdaBoostClassifier(n_estimators=100)
    
    cv = StratifiedKFold(n_splits=10)

    fscores = []
    for train, test in cv.split(X, Y):
        model = alg.fit(X.iloc[train], Y[train])
        Y_pred = model.predict(X.iloc[test])
        fscore = f1_score(Y[test], Y_pred, average='weighted', labels=np.unique(Y[test]))
        fscores.append(fscore)
        print(fscore)
        
    importances = list(model.feature_importances_)
    column_headers = list(df.columns.values)
    dicy = dict(zip(importances,column_headers))
    dicysort = collections.OrderedDict(sorted(dicy.items()))
    sns.set(style='whitegrid')
    ax = sns.barplot(x="day", y="total_bill", data=dict(dicysort))




    print('Average F-measure:', sum(fscores) / len(fscores))

Executing...
0.9932811738667893
0.9944991630100894
0.9938788002715931
0.9923290043290043
0.9940233126527541
0.9916154233363534
0.9934386723952305
0.9950282648360711
0.9930001940304126
0.9952674292119769


ValueError: Could not interpret input 'day'